In [8]:
# dep
! pip install -qU langchain-openai langchain langchain_community langchainhub
# vector database
! pip install chromadb==0.5.3

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
  Using cached https://mirrors.aliyun.com/pypi/packages/3e/c9/d55602e07bb839c578f4698b92db7ea88c1aa92f69d4c86076c1597e451b/chromadb-0.5.3-py3-none-any.whl (559 kB)
  Using cached https://mirrors.aliyun.com/pypi/packages/91/fd/e4bda6228637ecae5732162b5ac2a5a822e2ba8e546eb4997cde51b231a3/build-1.2.2-py3-none-any.whl (22 kB)
  Using cached https://mirrors.aliyun.com/pypi/packages/c0/59/1224cbae62c7b84c84088cdf6c106b9b2b893783c000d22c442a1672bc75/chroma-hnswlib-0.7.3.tar.gz (31 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached https://mirrors.aliyun.com/pypi/packages/06/ab/a1f7eed031aeb1c406a6e9d45ca04bff401c8a25a30dd0e4fd2caae767c3/fast

In [9]:
# import pkg
from langchain import hub as langchain_hub
from langchain.schema import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.text_splitter import MarkdownHeaderTextSplitter
from langchain_openai import OpenAIEmbeddings
import os
#from langchain_chroma import Chroma
from langchain_community.vectorstores.chroma import Chroma

In [10]:
# read ./data/data.md as knowledge
file_path = os.path.join('data', 'data.md')
with open(file_path, 'r', encoding='utf-8') as file:
    docs_string = file.read()

# Split the document into chunks base on markdown headers.
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]

# split text by header
text_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
splits = text_splitter.split_text(docs_string)
print("Length of splits: " + str(len(splits)))
print(splits)

Length of splits: 10
[Document(metadata={'Header 1': 'Payment System Operations Knowledge Base', 'Header 2': 'Table of Contents'}, page_content='1. [Introduction](#introduction)\n2. [System Architecture](#system-architecture)\n3. [Common Issues and Solutions](#common-issues-and-solutions)\n4. [Monitoring and Alerts](#monitoring-and-alerts)\n5. [Security Best Practices](#security-best-practices)\n6. [Backup and Recovery Procedures](#backup-and-recovery-procedures)\n7. [Performance Optimization](#performance-optimization)\n8. [Documentation and Resources](#documentation-and-resources)'), Document(metadata={'Header 1': 'Payment System Operations Knowledge Base', 'Header 2': 'Introduction'}, page_content='This knowledge base provides essential information for the operation and maintenance of the payment system. It is intended for system administrators, developers, and support staff.'), Document(metadata={'Header 1': 'Payment System Operations Knowledge Base', 'Header 2': 'System Architectu

In [11]:
# embed = vectorize & persist to db
embedding = OpenAIEmbeddings(
    model="text-embedding-3-small", 
    openai_api_key="BLOCKED",
    openai_api_base="https://api.apiyi.com/v1")
vectorstore = Chroma.from_documents(documents=splits, embedding=embedding,persist_directory="./chroma_langchain_db")
vectorstore.persist()

C:\Users\hiron\AppData\Local\Temp\ipykernel_15588\679650500.py:7: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [12]:
# retrieve by score
results = vectorstore.similarity_search_with_score(
    "payment service",
    k=1,
)
for res, score in results:
    print(f"* [similarity={score:3f}] {res.page_content} [{res.metadata}]")

* [similarity=1.060808] - **Components**: Describe the key components of the payment system, including:
- Payment Gateway
- Database
- Application Server
- Third-party Services  
- **Data Flow**: Outline the flow of data through the system from the user to the payment processor. [{'Header 1': 'Payment System Operations Knowledge Base', 'Header 2': 'System Architecture'}]


In [14]:
# retrieve by vector
results = vectorstore.similarity_search_by_vector_with_relevance_scores(
    embedding=embedding.embed_query("backup"), k=1
)
for doc, score in results:
    print(f"* [similarity={score:3f}] {doc.page_content} [{doc.metadata}]")

* [similarity=1.075319] - **Backup Schedule**: Define daily, weekly, and monthly backup strategies.
- **Recovery Plan**: Outline steps for restoring the system from backups in case of failure. [{'Header 1': 'Payment System Operations Knowledge Base', 'Header 2': 'Backup and Recovery Procedures'}]


In [17]:
# input to vector when retrieval
from openai import OpenAI

client = OpenAI(
    api_key="BLOCKED",
    base_url="https://api.apiyi.com/v1"
)

response = client.embeddings.create(
    input="backup",
    model="text-embedding-3-small"
)

print(response.data[0].embedding)

[0.01491538342088461, 0.003154066391289234, 0.04614410921931267, -0.0138293681666255, -0.025763986632227898, -0.006313909776508808, 0.011339619755744934, 0.012419858947396278, -0.04092198982834816, -0.02463175728917122, 0.004967943299561739, 0.023869235068559647, 0.005389641039073467, -0.0138293681666255, -0.004219863563776016, 0.0650222897529602, -0.04995671659708023, -0.03269600123167038, -0.03177173063158989, 0.0029980961699038744, -0.002538850298151374, 0.01320548728108406, 0.06728675216436386, -0.01329791359603405, -0.006400559563189745, -0.018727991729974747, -0.030616395175457, -0.05088098347187042, 0.04720701649785042, -0.07232401520013809, 0.02347642183303833, -0.04806196317076683, 0.021096430718898773, 0.004234305117279291, 0.03683210164308548, 0.03576919436454773, 0.038287825882434845, -0.025417385622859, 0.011899957433342934, -0.0018239861819893122, 0.003517997218295932, -0.05855241417884827, 0.034244149923324585, 0.037363555282354355, 0.03643928840756416, 0.040621604770421

In [18]:
# db as rag retriever
retriever = vectorstore.as_retriever()

# RAG Prompt from LangChain
# https://smith.langchain.com/hub/rlm/rag-prompt?organizationId=989ad331-949f-4bac-9694-660074a208a7
"""
You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:
"""
prompt = langchain_hub.pull("rlm/rag-prompt")

# print matched 
def format_docs(docs):
    print("matched：\n", "\n\n".join(doc.page_content for doc in docs))
    return "\n\n".join(doc.page_content for doc in docs)

llm = ChatOpenAI(
    model="gpt-4o-mini",
    openai_api_key="BLOCKED",
    openai_api_base="https://api.apiyi.com/v1",
    )

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# input to vector & compare in db
res = rag_chain.invoke("backup details")
print("\n\nLLM Answer：", res)

c:\Users\hiron\AppData\Local\Programs\Python\Python312\Lib\site-packages\langsmith\client.py:323: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


matched：
 - **Backup Schedule**: Define daily, weekly, and monthly backup strategies.
- **Recovery Plan**: Outline steps for restoring the system from backups in case of failure.

1. [Introduction](#introduction)
2. [System Architecture](#system-architecture)
3. [Common Issues and Solutions](#common-issues-and-solutions)
4. [Monitoring and Alerts](#monitoring-and-alerts)
5. [Security Best Practices](#security-best-practices)
6. [Backup and Recovery Procedures](#backup-and-recovery-procedures)
7. [Performance Optimization](#performance-optimization)
8. [Documentation and Resources](#documentation-and-resources)

- **Data Encryption**: Use TLS for data in transit and AES for data at rest.
- **Access Control**: Implement role-based access control (RBAC).
- **Regular Audits**: Conduct security audits and vulnerability assessments.

- **Load Testing**: Regularly perform load testing to identify bottlenecks.
- **Database Optimization**: Use indexing and query optimization techniques.


LLM A